In [ ]:
from abtem import GridScan, PixelatedDetector, Potential, Probe, show_atoms, SMatrix, AnnularDetector
from abtem.detect import PixelatedDetector
from abtem.reconstruct import MixedStatePtychographicOperator, RegularizedPtychographicOperator
from abtem.measure import Measurement, Calibration, bandlimit, center_of_mass
from abtem.utils import energy2wavelength
from ase.io import read
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
from pylab import cm
from matplotlib.widgets import RectangleSelector
import matplotlib.patches as pch
import tifffile
import tkinter.filedialog as tkf

In [ ]:
energy                            = 80E3
semiangle                         = 19.85182
step_size_real_space              = (0.40022, 0.40022)
reciprocal_space_sampling_mrad    = (1.640532, 1.640532)
center_x = 63.47938902629002
center_y = 62.62253022165034
file_adr = ""

In [ ]:
raw_data = tifffile.imread(file_adr)
print(raw_data.shape)

In [ ]:
x_cb_object = Calibration(offset=0, sampling=step_size_real_space[0], units="Å", name="x")
y_cb_object = Calibration(offset=0, sampling=step_size_real_space[1], units="Å", name="y")
dx_cb_object = Calibration(offset=-center_x*reciprocal_space_sampling_mrad[0], 
                           sampling=reciprocal_space_sampling_mrad[0], units="mrad", name="alpha_x")
dy_cb_object = Calibration(offset=-center_y*reciprocal_space_sampling_mrad[1], 
                           sampling=reciprocal_space_sampling_mrad[1], units="mrad", name="alpha_y")

experimental_measurement = Measurement(raw_data, calibrations=[x_cb_object, y_cb_object, dx_cb_object, dy_cb_object])
adf                      = AnnularDetector(inner=semiangle,outer=reciprocal_space_sampling_mrad[0]*raw_data.shape[2]/2).integrate(experimental_measurement)
print(semiangle, reciprocal_space_sampling_mrad[0]*raw_data.shape[2]/2)
pacbed                   = experimental_measurement.mean(axis=(0,1))

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(8,6))

adf.show(ax=ax1,title='ADF image')
pacbed.show(power=0.5,ax=ax2, cmap='inferno',title='PACBED pattern')

fig.tight_layout()
plt.show()

In [ ]:
print(experimental_measurement.shape)
print(*experimental_measurement.calibration_limits, sep="\n")
for i in range(experimental_measurement.dimensions):
    print(experimental_measurement.calibrations[i].name, experimental_measurement.calibrations[i].units, experimental_measurement.calibrations[i].sampling)

In [ ]:
experimental_measurement_sampling = tuple(energy2wavelength(energy)*1000/(cal.sampling * pixels) 
                                                      for cal,pixels 
                                                      in zip(experimental_measurement.calibrations[-2:], 
                                                             experimental_measurement.shape[-2:]))

print(f'pixelated_measurement sampling: {experimental_measurement_sampling}')

In [ ]:
experimental_measurement_extent = tuple(sampling*pixels for sampling,pixels 
                                      in zip(experimental_measurement_sampling, experimental_measurement.shape[-2:]))

print(f'pixelated_measurement extent: {experimental_measurement_extent}')

In [ ]:
probe_roi_limited    = Probe(semiangle_cutoff=semiangle, energy=energy,
                             sampling = experimental_measurement_sampling,
                             gpts     = experimental_measurement.shape[-2:])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
experimental_measurement.mean(axis=(0, 1)).show(power=0.5,cmap='inferno',ax=ax1);
probe_roi_limited.show(ax=ax2, power=0.5)
fig.tight_layout()
plt.show()

In [ ]:
bf_measurement = bandlimit(experimental_measurement, semiangle)
bf_measurement.show(cmap="inferno", power=0.5)
plt.show()

In [ ]:
com_x, com_y = center_of_mass(bf_measurement)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

com_x.show(ax=ax1)
com_y.show(ax=ax2)
fig.tight_layout()
plt.show()

In [ ]:
icom = center_of_mass(bf_measurement, return_icom=True)

icom.show()
plt.show()

In [ ]:
n_iter = 10
experimental_ptycho_operator = RegularizedPtychographicOperator(experimental_measurement,
                                                               semiangle_cutoff=semiangle,
                                                               energy=energy,
                                                               parameters={'object_px_padding':(16, 16)}).preprocess()

exp_objects, exp_probes, exp_positions, exp_sse  = experimental_ptycho_operator.reconstruct(
    max_iterations = n_iter,
    random_seed=1,
    return_iterations=True,
    parameters={'alpha':1.0,
                'beta':1.0})

print(exp_objects[-1].shape)
for i in range(exp_objects[-1].dimensions):
    print(exp_objects[-1].calibrations[i].name, exp_objects[-1].calibrations[i].units,
          exp_objects[-1].calibrations[i].sampling)

In [ ]:
plot_every = int(n_iter/5)

fig, axes = plt.subplots(2, int(np.ceil(len(exp_objects) / plot_every))+1, figsize=(20, 8))

for i, j in enumerate(range(0,len(exp_objects), plot_every)):
    axes[0,i].imshow(np.angle(exp_objects[j].array).T, origin='lower', cmap='gray')
    axes[0,i].set_title('iteration: %d, SSE: %.2e'%(j+1, exp_sse[j]))
    axes[1,i].imshow(np.abs(exp_probes[j].array).T, origin='lower', cmap='gray')
    axes[0,i].axis("off")
    axes[1,i].axis("off")

axes[0,-1].imshow(np.angle(exp_objects[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].set_title('iteration: %d, SSE: %.2e'%(n_iter, exp_sse[-1]))
axes[1,-1].imshow(np.abs(exp_probes[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].axis("off")
axes[1,-1].axis("off")

fig.tight_layout()
plt.show()

In [ ]:
fig, axd = plt.subplots(1, 2, figsize=(20, 10))
exp_objects[-1].angle().show(ax=axd[0], title=f"SSE = {float(exp_sse[-1]):.3e}", cmap='inferno')
exp_probes[-1].intensity().show(ax=axd[1], cmap="gray", power=0.5)
fig.tight_layout()
plt.show()